# Exploración preliminar y transformación de datos

In [1]:
# Bibliotecas requeridas
import sys
sys.path.append('../scr/')

import loadAndTransformData as ltd
import viewDataFunctions as vdf


import pandas as pd
import os


In [2]:
# Carga de datos
data = ltd.cargar_csv('C:/Users/jpmog/Downloads/tu_(1)_2024_08_30.csv')

Archivo cargado correctamente.


In [3]:
data.shape

(1081702, 11)

In [5]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1081702 entries, 0 to 1081701
Data columns (total 11 columns):
 #   Column                Non-Null Count    Dtype  
---  ------                --------------    -----  
 0   fecha                 1081702 non-null  object 
 1   codigo_factura        1081702 non-null  int64  
 2   mnt_total_a_pagar     1081702 non-null  float64
 3   cant_items            1081702 non-null  int64  
 4   familia               1078892 non-null  object 
 5   categoria             1081699 non-null  object 
 6   subcategoria          1081699 non-null  object 
 7   codigo_producto       1081699 non-null  float64
 8   cantidad_vendida      1081700 non-null  float64
 9   venta_bruta_producto  1081700 non-null  float64
 10  venta_neta_producto   1081699 non-null  float64
dtypes: float64(5), int64(2), object(4)
memory usage: 90.8+ MB


In [6]:
#ltd.dividir_y_guardar_csv(data, 'fecha', 'mes', '../data/raw/', 'Minimarket_Sales_Data')

In [7]:
# Cargar datos disponibles en el directorio de datos del proyecto
data2 = ltd.combinar_csv_por_prefijo('../data/raw/', 'Minimarket_Sales_Data')

In [12]:
data2.shape


(1081702, 11)

In [ ]:
model_data_producto = ltd.pivotear_df(data2, 'codigo_factura', 'codigo_producto')
model_data_familia = ltd.pivotear_df(data2, 'codigo_factura', 'familia')
model_data_categoria = ltd.pivotear_df(data2, 'codigo_factura', 'categoria')
model_data_subcategoria = ltd.pivotear_df(data2, 'codigo_factura', 'subcategoria')

In [ ]:
print(model_data_producto.shape, model_data_familia.shape, model_data_categoria.shape, model_data_subcategoria.shape)

In [ ]:
guardar_csv(model_data_producto, '../data/raw/', 'Minimarket_Sales_Data_Transform_By_Product')
guardar_csv(model_data_familia, '../data/raw/', 'Minimarket_Sales_Data_Transform_By_Family')
guardar_csv(model_data_categoria, '../data/raw/', 'Minimarket_Sales_Data_Transform_By_Category')
guardar_csv(model_data_subcategoria, '../data/raw/', 'Minimarket_Sales_Data_Transform_By_Subcategory')

In [ ]:
df_pivot = pd.crosstab(data2['codigo_factura'], data2['codigo_producto'])

In [ ]:
df_pivot